In [4]:
from kubernetes import client, config
config.load_config()

In [22]:
def get_virtualservice(name, namespace='default'):
    apiCustObject = client.CustomObjectsApi() # Patch virtual service
    params = dict(
        group ='networking.istio.io',
        version = 'v1beta1',
        namespace = namespace,
        plural='virtualservices',
        name = name
    )
    policy = apiCustObject.get_namespaced_custom_object(**params)
    return policy

def patch_virtualservice(service):
    apiCustObject = client.CustomObjectsApi() # Patch authorisation policy
    name = service['metadata']['name']
    namespace = service['metadata']['namespace']

    params = dict(
        group ='networking.istio.io',
        version = 'v1beta1',
        namespace = namespace,
        plural='virtualservices',
        name = name
    )

    patch_params = dict(body = service, **params)
    apiCustObject.patch_namespaced_custom_object(**patch_params)
    # Read after patch
    return apiCustObject.get_namespaced_custom_object(**params)

def traffic_shifting(name, weights, namespace='default'):
    service = get_virtualservice(name, namespace)
    http_route = service['spec']['http'][0]['route']
    for route in http_route:
      dest = route['destination']
      full_name = f"{dest['host']}_{dest['subset']}"
      if full_name in weights:
        route['weight'] = weights[full_name]

    patch_virtualservice(service)


In [23]:
traffic_shifting('reviews', {'reviews_v1': 70,'reviews_v3': 30 })

In [6]:
get_virtualservice('reviews')

{'apiVersion': 'networking.istio.io/v1beta1',
 'kind': 'VirtualService',
 'metadata': {'annotations': {'kubectl.kubernetes.io/last-applied-configuration': '{"apiVersion":"networking.istio.io/v1alpha3","kind":"VirtualService","metadata":{"annotations":{},"name":"reviews","namespace":"default"},"spec":{"hosts":["reviews"],"http":[{"route":[{"destination":{"host":"reviews","subset":"v1"},"weight":50},{"destination":{"host":"reviews","subset":"v3"},"weight":50}]}]}}\n'},
  'creationTimestamp': '2022-05-09T03:15:08Z',
  'generation': 4,
  'managedFields': [{'apiVersion': 'networking.istio.io/v1alpha3',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:kubectl.kubernetes.io/last-applied-configuration': {}}},
     'f:spec': {'.': {}, 'f:hosts': {}}},
    'manager': 'kubectl-client-side-apply',
    'operation': 'Update',
    'time': '2022-05-09T03:15:08Z'},
   {'apiVersion': 'networking.istio.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'field